# Report ISJEAAI 

imports

In [3]:
import re
import json

In [4]:
REPORT_FILE_PATH = './report_final.json'

In [5]:
def reload():
    try:
        with open(REPORT_FILE_PATH) as json_file:
            data = json.load(json_file)
            data = {k: v for k, v in sorted(data.items(), key=lambda item: item[0])}
        return data
    except:
        print("An exception occurred when trying to open %s" % REPORT_FILE_PATH)
        return []

## Utils

In [6]:
def get_experiment_metadata(key):
    pattern = r"^(\w+)_input_(\w+)_classes_(\w+)_nn_(\w+)$"
    match = re.match(pattern, key)
    if match:
        return [match[1], match[2], match[3], match[4]]
    return [None, None, None, None]

In [32]:
def generate_table(target_dataset):
    data = reload()
    table = '<html> <table> <tr>'
    #table += '<th> dataset </th>'
    table += '<th> model </th>'
    table += '<th> pages </th>'
    table += '<th> output </th>'
    table += '<th> val accuracy </th>'    
    table += '<th> val kappa </th>'        
    #table += '<th> test accuracy </th>'    
    #table += '<th> test kappa </th>'            
    #table += '<th> f1-score </th>'
    #table += '<th> f1 ND </th>'
    #table += '<th> f1 SD </th>'    
    table += '</tr>'

    max_acc = -1
    max_kappa = -1    
    max_t_kappa = -1
    max_t_acc = -1    
    max_f1_score = -1
    max_f1_score_FirstPage = -1
    max_f1_score_NextPage = -1

    max_t_kappa_exp = None
    max_t_acc_exp = None    
    max_acc_exp = None
    max_kappa_exp = None
    max_f1_score_exp = None
    max_f1_score_FirstPage_exp = None
    max_f1_score_NextPage_exp = None
    
    for key, value in data.items():
        
        dataset, pages, classes, model = get_experiment_metadata(key)       
        if dataset != target_dataset:
            continue
        f1_score = 100*value["weighted avg"]["f1-score"]
        f1_score_FirstPage = 100*value['FirstPage']['f1-score']
        f1_score_NextPage = 100*value['NextPage']['f1-score']        
        acc = 100*value["valid_acc"]
        kappa = 100*value["valid_kappa"]
        tacc = 100*value["test_acc"]
        tkappa = 100*value["test_kappa"]        

        if f1_score_FirstPage > max_f1_score_FirstPage:
            max_f1_score_FirstPage = f1_score_FirstPage
            max_f1_score_FirstPage_exp = key
        if f1_score_NextPage > max_f1_score_NextPage:
            max_f1_score_NextPage = f1_score_NextPage
            max_f1_score_NextPage_exp = key
        if f1_score > max_f1_score:
            max_f1_score = f1_score
            max_f1_score_exp = key            
        if acc > max_acc:
            max_acc = acc
            max_acc_exp = key
        if kappa > max_kappa:
            max_kappa = kappa
            max_kappa_exp = key
        if tacc > max_t_acc:
            max_t_acc = tacc
            max_t_acc_exp = key
        if tkappa > max_t_kappa:
            max_t_kappa = tkappa
            max_t_kappa_exp = key

    for key, value in data.items():

        f1_score = 100*value["weighted avg"]["f1-score"]
        f1_score_FirstPage = 100*value['FirstPage']['f1-score']
        f1_score_NextPage = 100*value['NextPage']['f1-score']        
        acc = 100*value["valid_acc"]
        kappa = 100*value["valid_kappa"]
        tacc = 100*value["test_acc"]
        tkappa = 100*value["test_kappa"]        

        diff_f1 = max_f1_score - f1_score 
        dfff_f1_FirstPage = max_f1_score_FirstPage  - f1_score_FirstPage
        diff_f1_NextPage = max_f1_score_NextPage - f1_score_NextPage
        diff_acc = max_acc - acc
        diff_kappa = max_kappa - kappa
        diff_tacc = max_t_acc - tacc
        diff_tkappa = max_t_kappa - tkappa
        
        diff_f1 = f'({diff_f1:.1f})' if diff_f1 > 0 else ''
        dfff_f1_FirstPage = f'({dfff_f1_FirstPage:.1f})' if dfff_f1_FirstPage > 0 else ''
        diff_f1_NextPage = f'({diff_f1_NextPage:.1f})' if diff_f1_NextPage > 0 else ''
        diff_acc = f'({diff_acc:.1f})' if diff_acc > 0 else ''
        diff_kappa = f'({diff_kappa:.1f})' if diff_kappa > 0 else ''
        diff_tacc = f'({diff_tacc:.1f})' if diff_tacc > 0 else ''
        diff_tkappa = f'({diff_tkappa:.1f})' if diff_tkappa > 0 else ''
        

        dataset, pages, classes, model = get_experiment_metadata(key)       
        if dataset != target_dataset:
            continue

        table += '<tr>'
        #table += f'<td> {dataset} </td>'
        table += f'<td> {model} </td>'
        table += f'<td> {pages} </td>'
        table += f'<td> {classes} </td>'
        table += '<td>' + ('<strong>' if max_acc_exp == key else '' ) + f'{acc:.1f}{diff_acc}%'  + ('</strong>' if max_acc_exp == key else '') +'</td>'            
        table += '<td>' + ('<strong>' if max_kappa_exp == key else '' ) + f'{kappa:.1f}{diff_kappa}%' + ('</strong>' if max_kappa_exp == key else '') +'</td>'        
        #table += '<td>' + ('<strong>' if max_t_acc_exp == key else '' ) + f'{tacc:.1f}{diff_tacc}%'  + ('</strong>' if max_t_acc_exp == key else '') +'</td>'            
        #table += '<td>' + ('<strong>' if max_t_kappa_exp == key else '' ) + f'{tkappa:.1f}{diff_tkappa}%' + ('</strong>' if max_t_kappa_exp == key else '') +'</td>'                
        #table += '<td>' + ('<strong>' if max_f1_score_exp == key else '' ) + f'{f1_score:.1f}{diff_f1}%' + ('</strong>' if max_f1_score_exp == key else '') +'</td>'        
        #table += '<td>' + ('<strong>' if max_f1_score_FirstPage_exp == key else '' ) + f'{f1_score_FirstPage:.1f}{dfff_f1_FirstPage}%' + ('</strong>' if max_f1_score_FirstPage_exp == key else '') +'</td>'        
        #table += '<td>' + ('<strong>' if max_f1_score_NextPage_exp == key else '' ) + f'{f1_score_NextPage:.1f}{diff_f1_NextPage}%' + ('</strong>' if max_f1_score_NextPage_exp == key else '') +'</td>'                
        table += '</tr>'  

    table+= '</table></html>'
    return table

In [34]:
print(generate_table('tobacco800'))

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> val accuracy </th><th> val kappa </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>80.4(0.9)%</td><td>54.8(9.5)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>67.4(13.8)%</td><td>25.2(39.1)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>79.9(1.3)%</td><td>52.8(11.5)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>74.1(7.1)%</td><td>44.8(19.5)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td>79.9(1.3)%</td><td>56.2(8.1)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>71.4(9.8)%</td><td>42.9(21.4)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 4 </td><td><strong>81.2%</strong></td><td><strong>64.3%</strong></td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 4 </td><td>76.8(4.5)%</td><td>51.6(12.7)%</td></tr></table></html>


## Tobacco800 - Resultado


## VALIDATION

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> val accuracy </th><th> val kappa </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>80.4(0.9)%</td><td>54.8(9.5)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>67.4(13.8)%</td><td>25.2(39.1)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>79.9(1.3)%</td><td>52.8(11.5)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>74.1(7.1)%</td><td>44.8(19.5)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td>79.9(1.3)%</td><td>56.2(8.1)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>71.4(9.8)%</td><td>42.9(21.4)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 4 </td><td><strong>81.2%</strong></td><td><strong>64.3%</strong></td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 4 </td><td>76.8(4.5)%</td><td>51.6(12.7)%</td></tr></table></html>

## test set
<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> test accuracy </th><th> test kappa </th><th> f1-score </th><th> f1 ND </th><th> f1 SD </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>87.8(4.2)%</td><td>74.5(9.1)%</td><td>86.3(4.8)%</td><td>89.0(3.7)%</td><td>82.6(6.7)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>69.2(22.8)%</td><td>38.8(44.7)%</td><td>69.7(21.4)%</td><td>71.3(21.4)%</td><td>67.5(21.8)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>89.6(2.4)%</td><td>78.3(5.3)%</td><td>88.3(2.8)%</td><td>90.4(2.2)%</td><td>85.3(4.0)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>79.9(12.2)%</td><td>58.2(25.4)%</td><td>77.5(13.6)%</td><td>80.9(11.7)%</td><td>72.9(16.4)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td><strong>92.0%</strong></td><td><strong>83.6%</strong></td><td><strong>91.1%</strong></td><td>92.4(0.2)%</td><td><strong>89.3%</strong></td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>83.7(8.3)%</td><td>67.0(16.5)%</td><td>81.9(9.2)%</td><td>84.3(8.4)%</td><td>78.5(10.8)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 4 </td><td>92.0%</td><td>83.4(0.2)%</td><td>91.1(0.0)%</td><td><strong>92.7%</strong></td><td>88.8(0.5)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 4 </td><td>86.1(5.9)%</td><td>70.2(13.3)%</td><td>84.5(6.6)%</td><td>87.2(5.5)%</td><td>80.6(8.7)%</td></tr></table></html>

In [66]:
print(generate_table('ai_lab_splitter'))

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> test accuracy </th><th> test kappa </th><th> f1-score </th><th> f1 FP </th><th> f1 NP </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>0.816</td><td>0.539</td><td>0.799</td><td>0.595</td><td>0.857</td></tr><tr><td> effnetB0_POS_11Block </td><td> 1 </td><td> 2 </td><td>0.929</td><td>0.715</td><td>0.930</td><td>0.805</td><td>0.957</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>0.958</td><td>0.837</td><td>0.959</td><td>0.889</td><td>0.974</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>0.954</td><td>0.825</td><td>0.954</td><td>0.871</td><td>0.972</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td><strong>0.966</strong></td><td>0.850</td><td>0.966</td><td>0.905</td><td><strong>0.979</strong></td></tr><tr><td> effnetB0_14_FREEZE_BATCH </td><td> 3 </td><td> 2 </td><td>0.957</td><td>0.825</td><td>0.957</td><td>0.879</td><td>0.974</td></tr><tr><td> VGG16_PRE_REVIEWLABELSPLIT </td><td> 

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> test accuracy </th><th> test kappa </th><th> f1-score </th><th> f1 ND </th><th> f1 SD </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>87.8(4.2)%</td><td>74.5(9.1)%</td><td>86.3(4.8)%</td><td>89.0(3.7)%</td><td>82.6(6.7)%</td></tr><tr><td> effnetB0_14_FREEZE_BATCH_PRE_VALID </td><td> 1 </td><td> 2 </td><td>69.2(22.8)%</td><td>38.8(44.7)%</td><td>69.7(21.4)%</td><td>71.3(21.4)%</td><td>67.5(21.8)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>89.6(2.4)%</td><td>78.3(5.3)%</td><td>88.3(2.8)%</td><td>90.4(2.2)%</td><td>85.3(4.0)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>79.9(12.2)%</td><td>58.2(25.4)%</td><td>77.5(13.6)%</td><td>80.9(11.7)%</td><td>72.9(16.4)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td><strong>92.0%</strong></td><td><strong>83.6%</strong></td><td><strong>91.1%</strong></td><td>92.4(0.2)%</td><td><strong>89.3%</strong></td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>83.7(8.3)%</td><td>67.0(16.5)%</td><td>81.9(9.2)%</td><td>84.3(8.4)%</td><td>78.5(10.8)%</td></tr><tr><td> VGG16_PRE_SHUFFLEOFF </td><td> 3 </td><td> 4 </td><td>92.0%</td><td>83.4(0.2)%</td><td>91.1(0.0)%</td><td><strong>92.7%</strong></td><td>88.8(0.5)%</td></tr><tr><td> effnetB0_FT_Layer11 </td><td> 3 </td><td> 4 </td><td>86.1(5.9)%</td><td>70.2(13.3)%</td><td>84.5(6.6)%</td><td>87.2(5.5)%</td><td>80.6(8.7)%</td></tr></table></html